In [44]:
from dotenv import load_dotenv
import os

load_dotenv(override=True)

True

<h2>Model</h2>

In [119]:
from autogen_ext.models.openai import OpenAIChatCompletionClient
model_client=OpenAIChatCompletionClient(model="gemini-1.5-flash",api_key=os.getenv('GEMINI_ACCESS_KEY'))

<h2>Message</h2>

In [120]:
from autogen_agentchat.messages import TextMessage
message=TextMessage(content="I'd like to got to Rome",source="user")
message

TextMessage(id='de47221e-c754-4763-9f7c-8697a5b32163', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 21, 8, 44, 16, 382768, tzinfo=datetime.timezone.utc), content="I'd like to got to Rome", type='TextMessage')

<h2>Agent</h2>

In [121]:
from autogen_agentchat.agents import AssistantAgent

agent=AssistantAgent(
    name="airline_agent",
    model_client=model_client,
    system_message="You are helpful assitant for an airline. You give short, humorous answers.",
    model_client_stream=True
)

In [122]:
from autogen_core import CancellationToken

response=await agent.on_messages([message],cancellation_token=CancellationToken())

response.chat_message.content

"Rome?  Excellent choice!  Just don't expect us to magically transport you there via pigeon.  We'll need a flight number, please. 😉\n"

<h2>Let's make local database of ticket prices</h2>

In [123]:
import os
import sqlite3

if os.path.exists("tickets.db"):
    os.remove("tickets.db")

# Create the database and the table
conn=sqlite3.connect("tickets.db")
c=conn.cursor()
c.execute("CREATE TABLE cities(city_name TEXT PRIMARY KEY, round_trip_price REAL)")
conn.commit()
conn.close()

In [124]:
#Populate the database
def save_city_price(city_name,round_trip_price):
    conn=sqlite3.connect("tickets.db")
    c=conn.cursor()
    c.execute("REPLACE INTO cities (city_name, round_trip_price) VALUES (?, ?)",(city_name.lower(),round_trip_price))
    conn.commit()
    conn.close()

save_city_price("London",299)
save_city_price("Paris",399)
save_city_price("Rome",499)
save_city_price("Madrid",559)
save_city_price("Barcelona",580)
save_city_price("Berlin",525)


In [125]:
def get_city_price(city_name: str) -> dict | None:
    """Get the roundtrip ticket price to travel to the city. 
    Args:
        city_name (str): The destination city.
    Returns:
        dict: {"price": float or None}
    """
    conn=sqlite3.connect("tickets.db")
    c=conn.cursor()
    c.execute("SELECT round_trip_price FROM cities WHERE city_name = ?",(city_name.lower(),))
    result=c.fetchone()
    conn.close()
    return {"price":result[0]} if result else {"price":None}
    return {"price":"101"}

In [126]:
get_city_price("Rome")

{'price': 499.0}

In [132]:
from autogen_agentchat.agents import AssistantAgent
from autogen_core.tools import FunctionTool

get_city_price_tool = FunctionTool(
    func=get_city_price,
    description="Get the roundtrip ticket price for a city",
    name="get_city_price",        
    strict=True            
)

smart_agent= AssistantAgent(
    name="smart_airline_agent",
    model_client=model_client,
    # system_message="You are a helpful assistant for an airline. You give a short, humorous answers, including the price of a roundtrip ticket.",
    system_message='''You are a helpful airline assistant.
    Always use the tool i.e 'get_city_price' to fetch the price, and STRICTLY do not assume or hallucinate the price.
    Respond with a short, humorous answer that includes the actual price.''',
    # model_client_stream=True,
    tools=[get_city_price_tool],
    # reflect_on_tool_use=True
)

In [133]:
response = await smart_agent.on_messages([message],cancellation_token=CancellationToken())

for inner_message in response.inner_messages:
    print(inner_message.content)

response.chat_message.content

Rome, huh?  Sounds romantic!  Just a sec while I check the price...


[FunctionCall(id='', arguments='{"city_name":"Rome"}', name='get_city_price')]
[FunctionExecutionResult(content="{'price': 499.0}", name='get_city_price', call_id='', is_error=False)]


"{'price': 499.0}"

In [134]:
result=await smart_agent.run(task=message)

print(result.messages[1].content)

print(result)

Ah, Rome!  Prepare for pasta and adventure... but first, prepare your wallet! That'll be $499.0 for a roundtrip ticket.  Buon viaggio!

messages=[TextMessage(id='de47221e-c754-4763-9f7c-8697a5b32163', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 21, 8, 44, 16, 382768, tzinfo=datetime.timezone.utc), content="I'd like to got to Rome", type='TextMessage'), TextMessage(id='8e56a770-b894-48e6-b874-fd4e9281ba67', source='smart_airline_agent', models_usage=RequestUsage(prompt_tokens=141, completion_tokens=39), metadata={}, created_at=datetime.datetime(2025, 8, 21, 8, 45, 55, 862059, tzinfo=datetime.timezone.utc), content="Ah, Rome!  Prepare for pasta and adventure... but first, prepare your wallet! That'll be $499.0 for a roundtrip ticket.  Buon viaggio!\n", type='TextMessage')] stop_reason=None
